# Exploring Data (lesson 1)

In [1]:
#importing packages
from __future__ import print_function
import os
import re
import string
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from numpy import array
from pickle import dump
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

Using TensorFlow backend.


Starting of by downloading data and looking at the data, which we need to clean. We observe following
- Illustration texts
- The text file contain multiple stories, these should be split into multiple files in order for the model to know there is groupings (Extension: will be reviewed in lesson 5)
- The file contains a number of punctuations (,;.* etc.), which are irrelevant for the model
- Capital letters as headers

The input sequences are at first at 50 across sentences, chapters and stories. 

In [2]:
# creating function to load documents into memory
def load_doc(filename):
	file = open(filename, 'r', encoding="utf8") #loading an existing file
	text = file.read() #opening the file and assigning it to the variable text
	file.close() #close the file
	return text #output = text

In [3]:
#concatenate text files
os.chdir('C:\\Users\\soil\\Desktop\\Data')
 
files = os.listdir()

filenames = ['22693-0.txt', '3454-0.txt', '52521.txt', '5615-0.txt']

with open('adventures.txt', 'w', encoding="utf8") as outfile:
    for fname in filenames:
        with open(fname, encoding="utf8") as infile:
            outfile.write(infile.read())

In [4]:
# load document
in_filename = 'adventures.txt'#specifying the filename of the data we wish to load
doc = load_doc(in_filename) #loading the file
print(doc[:10000]) #printing the first 200 characters of the loading document

﻿Just as a little child holds out its hands to catch the sunbeams, to
feel and to grasp what, so its eyes tell it, is actually there, so,
down through the ages, men have stretched out their hands in eager
endeavour to know their God. And because only through the human was
the divine knowable, the old peoples of the earth made gods of their
heroes and not unfrequently endowed these gods with as many of the
vices as of the virtues of their worshippers. As we read the myths of
the East and the West we find ever the same story. That portion of the
ancient Aryan race which poured from the central plain of Asia,
through the rocky defiles of what we now call "The Frontier," to
populate the fertile lowlands of India, had gods who must once have
been wholly heroic, but who came in time to be more degraded than the
most vicious of lustful criminals. And the Greeks, Latins, Teutons,
Celts, and Slavonians, who came of the same mighty Aryan stock, did
even as those with whom they owned a common anc

# Cleaning data (lesson 2)

In lesson 1, we observed several potential issues in the data, which we need to remove from the data. This is done below:

In [5]:
#removing illustration descriptions
doc = re.sub(r'\[[^)]*\]', '', doc) #using re.sub function and regular expressions 
#[ - an opening bracket 
#[^()]* - zero or more characters other than those defined, that is, any characters other than [ and ]
#\] - a closing bracket


In [6]:
#removing headers
doc = re.sub(r'[A-Z]{2,}','', doc) #replacing capital letters longer than 1 with nothing. 


In [7]:
#removing special characters, since one of the deliminators
#doc = doc.replace("'", '')

#deliminators 
#deliminators = c('THE FIR TREE', 'LITTLE TUK', 'THE UGLY DUCKLING', 'LITTLE IDA'S FLOWERS', 'THE STEADFAST TIN SOLDIER, LITTLE THUMBELINA, SUNSHINE STORIES', 'THE DARNING-NEEDLE', 'THE LITTLE MATCH GIRL', 'THE LOVING PAIR', 'THE LEAPING MATCH', 'THE HAPPY FAMILY, THE GREENIES, 'OLE-LUK-OIE', 'THE DREAM GOD', 'THE MONEY BOX', 'ELDER-TREE MOTHER', 'THE SNOW QUEEN', 'THE ROSES AND THE SPARROWS', 'THE OLD HOUSE', 'THE CONCEITED APPLE BRANCH' 

#splitting the file into strings consisting of one adventure
#doc = doc.split ()

In [8]:
# turn a doc into clean tokens
def clean_doc(doc): #making a function
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token - https://www.geeksforgeeks.org/python-maketrans-translate-functions/
	table = str.maketrans('', '', string.punctuation) #Third argument specifies the wished deleted items
	tokens = [w.translate(table) for w in tokens] #translate applies the translation table applied on the looping through the tokens list
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()] #Replace word with word, if the word is alphabetic
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [9]:
tokens = clean_doc(doc) #using the clean function to cleaning our document
print(tokens[:200])
print('Total Tokens: %d' % len(tokens)) #printing the number of tokens
print('Unique Tokens: %d' % len(set(tokens))) #printing the number of unique tokens by grouping similair tokens

['as', 'a', 'little', 'child', 'holds', 'out', 'its', 'hands', 'to', 'catch', 'the', 'sunbeams', 'to', 'feel', 'and', 'to', 'grasp', 'what', 'so', 'its', 'eyes', 'tell', 'it', 'is', 'actually', 'there', 'so', 'down', 'through', 'the', 'ages', 'men', 'have', 'stretched', 'out', 'their', 'hands', 'in', 'eager', 'endeavour', 'to', 'know', 'their', 'god', 'and', 'because', 'only', 'through', 'the', 'human', 'was', 'the', 'divine', 'knowable', 'the', 'old', 'peoples', 'of', 'the', 'earth', 'made', 'gods', 'of', 'their', 'heroes', 'and', 'not', 'unfrequently', 'endowed', 'these', 'gods', 'with', 'as', 'many', 'of', 'the', 'vices', 'as', 'of', 'the', 'virtues', 'of', 'their', 'worshippers', 'as', 'we', 'read', 'the', 'myths', 'of', 'the', 'east', 'and', 'the', 'west', 'we', 'find', 'ever', 'the', 'same', 'story', 'that', 'portion', 'of', 'the', 'ancient', 'aryan', 'race', 'which', 'poured', 'from', 'the', 'central', 'plain', 'of', 'asia', 'through', 'the', 'rocky', 'defiles', 'of', 'what', 'w

In [10]:
# organize into sequences of tokens
length = 50 + 1 #defining the length of the sequence
sequences = list() #creating an empty list
for i in range(length, len(tokens)): #range takes two arguments: start point and end point
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq) #-join returns a string in which the string elements of sequence have been joined by str separator
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 178753


In [11]:
# save tokens to file, one dialog per line
def save_doc(lines, filename): #creating a saving function
	data = '\n'.join(lines) #joining the lines seperated by 'enter'
	file = open(filename, 'w', encoding="utf8") #create empty new file
	file.write(data) #input the data in the empty file
	file.close() #close the file

In [12]:
# save sequences to file
out_filename = 'fairytales_sequences.txt'
save_doc(sequences, out_filename)

# Training the model (Lesson 3)

In [13]:
# loading the cleaned data
in_filename = 'fairytales_sequences.txt'
doc = load_doc(in_filename)

#split the text by lineshifts creating a list of 51 items long string
lines = doc.split('\n')

In [14]:
# integer encode sequences of words
tokenizer = Tokenizer() #assign the tokenizer function to a variable
tokenizer.fit_on_texts(lines) #function of keras finds all of the unique words in the data and assigns each a unique integer
sequences = tokenizer.texts_to_sequences(lines) #translating the input lines into integers
tokenizer.word_index #checking the dictionary of the transformed wordsb

{'the': 1,
 'and': 2,
 'to': 3,
 'he': 4,
 'of': 5,
 'a': 6,
 'was': 7,
 'in': 8,
 'that': 9,
 'his': 10,
 'it': 11,
 'she': 12,
 'her': 13,
 'had': 14,
 'you': 15,
 'him': 16,
 'as': 17,
 'for': 18,
 'with': 19,
 'they': 20,
 'but': 21,
 'on': 22,
 'i': 23,
 'said': 24,
 'at': 25,
 'so': 26,
 'all': 27,
 'not': 28,
 'when': 29,
 'be': 30,
 'is': 31,
 'have': 32,
 'then': 33,
 'one': 34,
 'were': 35,
 'out': 36,
 'there': 37,
 'them': 38,
 'who': 39,
 'this': 40,
 'will': 41,
 'up': 42,
 'little': 43,
 'from': 44,
 'my': 45,
 'could': 46,
 'would': 47,
 'no': 48,
 'into': 49,
 'man': 50,
 'by': 51,
 'king': 52,
 'came': 53,
 'their': 54,
 'which': 55,
 'me': 56,
 'if': 57,
 'went': 58,
 'what': 59,
 'very': 60,
 'did': 61,
 'down': 62,
 'now': 63,
 'do': 64,
 'old': 65,
 'day': 66,
 'are': 67,
 'your': 68,
 'before': 69,
 'where': 70,
 'time': 71,
 'must': 72,
 'only': 73,
 'again': 74,
 'back': 75,
 'about': 76,
 'more': 77,
 'go': 78,
 'himself': 79,
 'come': 80,
 'after': 81,
 'like

In [15]:
#calculating vocabulary size to estimate the size of the embedding layer
vocab_size = len(tokenizer.word_index) + 1 #since indexing of array are zero-offset, the index of the vocabulary must be one larger than the length
vocab_size

8767

In [16]:
# separate into input and output
sequences = array(sequences) #transforming the sequens of integers to arrays
X, y = sequences[:,:-1], sequences[:,-1] #defining X (input sequences) and y (output words)
y = to_categorical(y, num_classes=vocab_size) #to_categorical converts a class vector (integers) to binary class matrix
seq_length = X.shape[1] #gives you the dimension of the array, which we put to be 50
seq_length

50

We will use a two LSTM hidden layers with 100 memory cells each. More memory cells and a deeper network may achieve better results.

In [17]:
# define model
model = Sequential() #assigning the sequential function to a model
model.add(Embedding(vocab_size, 50, input_length=seq_length)) #defining embedding layer size
model.add(LSTM(100, return_sequences=True)) #adding layer of nodes
model.add(LSTM(100))  #adding layer of nodes
model.add(Dense(100, activation='relu')) #specifying the structure of the hidden layer, recu is an argument of a rectified linear unit. 
model.add(Dense(vocab_size, activation='softmax')) #using the softmax function to creating probabilities
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            438350    
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 8767)              885467    
Total params: 1,474,717
Trainable params: 1,474,717
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #the compile function configures the model for training specifying the categorical cross entropy loss
# fit model
#model.fit(X, y, batch_size=128, epochs=100) #training the model 

In [19]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

# Lesson 4

In [20]:
# load cleaned text sequences
doc = load_doc('fairytales_sequences.txt')
lines = doc.split('\n')

In [21]:
#Specifying input sequences length to prompt the model
seq_length = len(lines[3].split()) - 1 #splitting the sequences into words, counting them -1
seq_length

50

In [22]:
#loading model
model= load_model('model.h5')
tokenizer = load(open('tokenizer.pkl', 'rb')) #r for read

In [23]:
#selecting a seed text
seed_text = lines[randint(0,len(lines))] #returns random integer between 0 and how many lines there is, and indexes this. 
print(seed_text + '\n') #prints the selected text
len(seed_text.split())

and like a candle clear in this dark country of the world thou art my woe my early light my music dying stephen phillips then idas in the humility that comes from perfect love drooped low his head and was silent in silence for a minute stood the threea god a



51

In [24]:
#translating the input text to integers
encoded = tokenizer.texts_to_sequences([seed_text])[0]
encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

The model can predict the next word directly by calling model.predict_classes() that will return the index of the word with the highest probability

In [25]:
# predict probabilities for each word
yhat = model.predict_classes(encoded, verbose=1)

1/1 [==============================] - 0s 191ms/step


Looking up the index in the Tokenizers mapping to get the associated word

In [26]:
#making a loop to translate integer to word
out_word = ''
for word, index in tokenizer.word_index.items():
	if index == yhat:
		out_word = word
		break

out_word

'bullcalf'

The input sequences will get too long, in order to keep them to 50 items using the following function, which pads sequences to the same lengt

In [27]:
#Setting the max length to be 50 items by removing items from the beginnning of the sequence
#encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')

In [28]:
#creating a function, which generates the predicted output
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list() #make an empty list
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [29]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

#saving generated text
out_filename = 'first_output.txt'
save_doc(generated, out_filename)


bullcalf bullcalf bullcalf hearing hearing hearing hearing hearing pitifully pitifully pitifully pitifully pitifully pitifully sparrow liberty liberty liberty liberty gael rise translation translation translation summer bullcalf bullcalf bullcalf hearing hearing kindly hearing limpets universe summer universe summer quickset quickset quickset taverner taverner taverner bee bee spanned spanned bullcalf bullcalf roguishly roguishly roguishly plaiting clerk walked walked walked probing probing vineyards vineyards overhear violets violets excursion reveals eva eva iveragh stately stately stainless stainless stainless stainless boating boating refreshed liberty glimmer glimmer stir stir stir triton enclosure lurking lurking lurking lurking lurking surprises surprises sparrow sparrow roguishly roguishly roguishly clerk clerk


# Lesson 5: Advancing the model


Pre-Trained Word Embedding. Extend the model to use pre-trained word2vec or GloVe vectors to see if it results in a better model.

In [30]:
# load the whole embedding into memory
embeddings_index = dict()
os.chdir('C:\\Users\\soil\\Desktop')
f = open('glove.6B.100d.txt', encoding = 'utf8')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [31]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [32]:
encoded.shape

(1, 50)

In [33]:
# define model
model = Sequential() #assigning the sequential function to a model
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=seq_length, trainable = False)) #defining embedding layer size
model.add(LSTM(100, return_sequences=True)) #adding layer of nodes
model.add(LSTM(100))  #adding layer of nodes
#model.add(Flatten())
model.add(Dense(100, activation='relu')) #specifying the structure of the hidden layer, recu is an argument of a rectified linear unit. 
model.add(Dense(vocab_size, activation='softmax')) #using the softmax function to creating probabilities
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           876700    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 100)           80400     
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 8767)              885467    
Total params: 1,933,067
Trainable params: 1,056,367
Non-trainable params: 876,700
_________________________________________________________________
None


In [34]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #the compile function configures the model for training specifying the categorical cross entropy loss

# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# fit model
model.fit(X, y, batch_size=128, epochs=100) #training the model 

Epoch 1/100
178753/178753 [==============================] - 422s 2ms/step - loss: 6.2823 - acc: 0.0745
Epoch 2/100
178753/178753 [==============================] - 480s 3ms/step - loss: 5.9009 - acc: 0.0967
Epoch 3/100
178753/178753 [==============================] - 386s 2ms/step - loss: 5.6119 - acc: 0.1148
Epoch 4/100
178753/178753 [==============================] - 385s 2ms/step - loss: 5.4314 - acc: 0.1244
Epoch 5/100
178753/178753 [==============================] - 386s 2ms/step - loss: 5.2870 - acc: 0.1316
Epoch 6/100
178753/178753 [==============================] - 382s 2ms/step - loss: 5.1701 - acc: 0.1355
Epoch 7/100
178753/178753 [==============================] - 390s 2ms/step - loss: 5.0733 - acc: 0.1394
Epoch 8/100
178753/178753 [==============================] - 385s 2ms/step - loss: 4.9883 - acc: 0.1424
Epoch 9/100
178753/178753 [==============================] - 385s 2ms/step - loss: 4.9139 - acc: 0.1462
Epoch 10/100
178753/178753 [==============================] - 38

178753/178753 [==============================] - 386s 2ms/step - loss: 3.5004 - acc: 0.2598
Epoch 80/100
178753/178753 [==============================] - 383s 2ms/step - loss: 3.4918 - acc: 0.2598
Epoch 81/100
178753/178753 [==============================] - 391s 2ms/step - loss: 3.4816 - acc: 0.2609
Epoch 82/100
178753/178753 [==============================] - 387s 2ms/step - loss: 3.4725 - acc: 0.2631
Epoch 83/100
178753/178753 [==============================] - 390s 2ms/step - loss: 3.4610 - acc: 0.2643
Epoch 84/100
178753/178753 [==============================] - 385s 2ms/step - loss: 3.4544 - acc: 0.2654
Epoch 85/100
178753/178753 [==============================] - 384s 2ms/step - loss: 3.4469 - acc: 0.2661
Epoch 86/100
178753/178753 [==============================] - 382s 2ms/step - loss: 3.4371 - acc: 0.2678
Epoch 87/100
178753/178753 [==============================] - 384s 2ms/step - loss: 3.4286 - acc: 0.2696
Epoch 88/100
178753/178753 [==============================] - 386s 2

In [35]:
# save the model to file
model.save('model_advancedK.h5')
# save the tokenizer
dump(tokenizer, open('tokenizerK.pkl', 'wb'))

In [36]:
#loading model
model= load_model('model_advancedK.h5')
tokenizer = load(open('tokenizerK.pkl', 'rb')) #r for read

In [42]:
#selecting a seed text
seed_text = lines[randint(0,len(lines))] #returns random integer between 0 and how many lines there is, and indexes this. 
print(seed_text + '\n') #prints the selected text
len(seed_text.split())

while grimace was too much blinded by her fury to notice what was going on the princess was quickly soaring out of her reach all this time souci had been wandering through the world with his precious thread carefully fastened round him seeking every possible and impossible place where his beloved



51

In [43]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

#saving generated text
out_filename = 'K_output.txt'
save_doc(generated, out_filename)

grip of the carts of gold and the shells were beaten in the loneliness of the mistress singing a loud neck and thread forth on the immense ruins and plunged out against the trees and sang the gloomy schooner sobbing and knows fight to be the offering of the beasts and grapes i have been softened and i will never do to think of the king and the snowqueen stretch and steal on the ground and bites you wants you like a brushing and the jackal fetched me and i will go to the other said the king you must


Things to be consider to advance the model: 

- We could process the data so that the model only ever deals with self-contained sentences and pad or truncate the text to meet this requirement for each input sequence. You could explore this as an extension to this tutorial.
- MORE DATA!

